In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

// TODO : Find out how to restart kernel

In [5]:
%%plantuml 

@startuml
Alice -> Bob: Authentication Request
Bob --> Alice: Authentication Response
@enduml 

UsageError: Cell magic `%%plantuml` not found.


In [44]:
# Delegation pattern example
class Microwave:
    def __init__(self):
        pass

    def heat_up_food(self):
        print("Food is being microwaved")

    def boil_water(self):
        print("Water is being boiled")

    def ruin_cake(self):
        print("Cake is being ruined... ah come on...why even lol")

class Dishwasher:
    def __init__(self):
        pass

    def wash_dishes(self):
        print("Dishwasher starting")

    def leave_spots(self):
        print("Dishes have spots, have fun being embarassed in front of guests")

    def wash_clothes(self):
        print("Washing dish shaped clothes... wtf")

class Kitchen ():
    def __init__(self):
        self.microwave = Microwave()
        self.dishwasher = Dishwasher()

        def get_custom_methods(obj):
            return [f for f in dir(obj) if not f.startswith('_') and callable(getattr(obj, f))]

        # how to unit test not callable?
        self.microwave_methods = get_custom_methods(self.microwave)
        # checking for callable overhead
        self.dishwasher_methods = get_custom_methods(self.dishwasher)

    def __getattr__(self, func):
        # what is the diff between *args and **kwargs?
        def method(*args):
            if func in self.microwave_methods:
                return getattr(self.microwave, func)(*args)
            if func in self.dishwasher_methods:
                return getattr(self.dishwasher, func)(*args)
            else:
                raise AttributeError(f"Method {func} not found")
        return method


In [45]:
kitchen = Kitchen()
kitchen.wash_dishes()
kitchen.boil_water()
kitchen.ruin_cake()
kitchen.leave_spots()
kitchen.wash_clothes()

Dishwasher starting
Water is being boiled
Cake is being ruined... ah come on...why even lol
Dishes have spots, have fun being embarassed in front of guests
Washing dish shaped clothes... wtf


In [1]:
# Decorator pattern examples
from datetime import datetime

class Employee():
    def __init__(self, name, salary):
        self.name = name
        self.salary = salary

class SalaryYTDReport:
    def __init__(self, employees):
        self.employees = employees
        self.data = []

    def prep_data(self):
        self.data = []
        for employee in self.employees:
            emp_sal_dict = {'Employee': employee.name, 'YTD_salary':self.calculate_YTD_salary(employee.salary)}
            self.data.append(emp_sal_dict)

    def calculate_YTD_salary(self, salary):
        current_month = datetime.today().month
        ytd_salary = (salary/12) * current_month
        ytd_salary = round(ytd_salary, 2)
        return ytd_salary

In [12]:
class HTMLReportDecorator:
    def __init__(self, report):
        self.report = report
        self.html_report = []
        self.report_methods = [f for f in dir(SalaryYTDReport) if not f.startswith('_') and callable(getattr(SalaryYTDReport, f))]
        self.report_attributes = [a for a in report.__dict__.keys()]

    def __getattr__(self, func):
        if func in self.report_methods:
            def method(*args):
                # for functions, return AND call the function
                return getattr(self.report, func)(*args)
            return method
        elif func in self.report_attributes:
            # for attributes, just return the attribute
            return getattr(self.report, func)
        else:
            raise AttributeError

    def report_data(self):
        self.html_report = []
        self.prep_data()
        for row in self.data:  # type: ignore
            name = f"<b>{row['Employee']}</b>"
            ytd = f"<i>{row['YTD_salary']}</i>"
            html_row = f"{name}: {ytd}<br />"
            self.html_report.append(html_row)

In [14]:
empl1 = Employee('Mike', -47)
empl2 = Employee('Ike', 41)
empl3 = Employee('Scott', 38)
empl4 = Employee('Cathy', 44)

report = SalaryYTDReport([empl1, empl2, empl3, empl4])
report = HTMLReportDecorator(report)

for employee in report.employees:  # type: ignore
    print(employee.name)

report.report_data()
report.html_report


Mike
Ike
Scott
Cathy


['<b>Mike</b>: <i>-43.08</i><br />',
 '<b>Ike</b>: <i>37.58</i><br />',
 '<b>Scott</b>: <i>34.83</i><br />',
 '<b>Cathy</b>: <i>40.33</i><br />']